In [7]:
import io
import re
import string
import tensorflow as tf
import tqdm
import jieba
import jieba.analyse
import numpy as np
import functools
import os
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from gensim.models import word2vec
import gensim
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer, TfidfVectorizer
from sklearn.metrics import silhouette_score

In [8]:
stop_words = ["本文","出处","原文","文献号","页","号","分","类","名","出版","评价","作者","原刊","的","了","而","是","你","我"]
f1 = open("./scu_stopwords.txt","r",encoding="utf-8")
for line in f1.readlines():
    stop_words.append(line.split('\n')[0])

In [9]:
def cmp(x,y):
    if int(x[:-4])>int(y[:-4]):
        return 1
    else:
        return -1
    
ans = []
fname = []
path = "D:/workspace/THUCNews/THUCNews"
files = os.listdir(path)
print(files)
for f in files:
    name = []
    subpath = path+'/'+f
    for root,dirs,files in os.walk(subpath):
        index = 0
        print(index)
        files.sort(key=functools.cmp_to_key(cmp))
        for filename in files:
            fname = subpath+"/"+filename
            fn = open(fname,"r",encoding="utf-8")
            s = fn.read()
            seg = list(jieba.cut(s))   #精确模式
            seg1 = set('')
            for i in range(0,len(seg)):
                if u'\u4e00' <=seg[i]<= u'\u9fff':
                    try:
                        a = stop_words.index(seg[i])
                    except:
                        seg1.add(seg[i])
            ans.append({tuple(seg1):fname})
            index +=1
            if index == 1000:
                break
print(len(ans))

['体育', '娱乐', '家居', '彩票', '房产', '教育', '时尚', '时政', '星座', '游戏', '社会', '科技', '股票', '财经']
0
0
0
0
0
0
0
0
0
0
0
0
0
0
14000


In [10]:
tfidf = jieba.analyse.extract_tags
keywords = []
for i in ans:
    for j in i.keys(): 
        tt = tfidf((str(j)),topK=5,withWeight=True)
        for k in tt:
            keywords.append({k[0]:[i[j],k[1]]})
#         keywords.append(tt)

In [11]:
k ={}
for i in keywords:
    for j in i.keys():
        if j in k.keys():
            k[j].append(i[j])
        else:
            k.setdefault(j,[i[j]])
rk={}
for i in k.keys():
    if len(k[i]) > 2:
        rk.setdefault(i,k[i])
#         rk.append({i:k[i]})
# print(rk[:6])
print(len(rk))

5189


In [16]:
# print(rk[:10])

In [17]:
print(keywords[:10])
print(len(keywords))

[{'殷家': ['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.09015538988239437]}, {'国奥': ['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.08812945979507042]}, {'马晓旭': ['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.08418850354154929]}, {'转好': ['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.08418850354154929]}, {'傅亚雨': ['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.08418850354154929]}, {'潜滋暗长': ['D:/workspace/THUCNews/THUCNews/体育/1.txt', 0.047933371213793104]}, {'主场优势': ['D:/workspace/THUCNews/THUCNews/体育/1.txt', 0.047933371213793104]}, {'阿尔加': ['D:/workspace/THUCNews/THUCNews/体育/1.txt', 0.047933371213793104]}, {'逼抢': ['D:/workspace/THUCNews/THUCNews/体育/1.txt', 0.047933371213793104]}, {'失球': ['D:/workspace/THUCNews/THUCNews/体育/1.txt', 0.045543208522068966]}]
69927


In [18]:
text = {}
for d in keywords:
    for k in d.keys():
        text.setdefault(d[k][0],[])
for d in keywords:
    for k in d.keys():
        if k in rk.keys():
            if not d[k][0] in text.keys():
                text.setdefault(d[k][0],[[k,d[k][1]]])
            else:
                text[d[k][0]].append([k,d[k][1]])
print(len(text))
# print(text)

14000


In [20]:
index = 0
for key in text:
    index += 1
    print(key+':'+str(text[key]))
    if (index > 10):
        break

D:/workspace/THUCNews/THUCNews/体育/0.txt:[['国奥', 0.08812945979507042], ['马晓旭', 0.08418850354154929], ['转好', 0.08418850354154929]]
D:/workspace/THUCNews/THUCNews/体育/1.txt:[['主场优势', 0.047933371213793104], ['阿尔加', 0.047933371213793104], ['逼抢', 0.047933371213793104], ['失球', 0.045543208522068966]]
D:/workspace/THUCNews/THUCNews/体育/2.txt:[['运动队', 0.29796150692619044]]
D:/workspace/THUCNews/THUCNews/体育/3.txt:[['轮空', 0.12870997825925926], ['唐娜', 0.12870997825925926], ['女乒', 0.12870997825925926]]
D:/workspace/THUCNews/THUCNews/体育/4.txt:[]
D:/workspace/THUCNews/THUCNews/体育/5.txt:[['美国队', 0.3575538083114286], ['体育讯', 0.35117827827142856], ['外卡', 0.34156478579714283]]
D:/workspace/THUCNews/THUCNews/体育/6.txt:[['体育讯', 0.2508416273367347]]
D:/workspace/THUCNews/THUCNews/体育/7.txt:[['体育讯', 0.384101241859375]]
D:/workspace/THUCNews/THUCNews/体育/8.txt:[['队服', 0.04983973762792453], ['参透', 0.04722408789018868]]
D:/workspace/THUCNews/THUCNews/体育/9.txt:[['金牌得主', 0.06465431466046512], ['体育讯', 0.0571685569279069

In [21]:
print(ans[0])

{('奥运', '变得', '好', '外场', '保证', '引发', '见', '现在', '当天', '情况', '走', '当', '雨', '一周', '国奥队', '对', '显示', '点', '酒店', '娇贵', '主要', '马晓旭', '几个', '参加', '再出', '转好', '大雨', '疾病', '减员', '碰到', '队员', '全队', '下', '跟', '正式', '迹象', '发烧', '慢跑', '干扰', '试一试', '傅亚雨', '天气', '困扰', '球队', '来说', '再度', '塞尔维亚', '到', '日常', '沈阳', '再', '都', '之后', '意思', '球员', '来', '这种', '立即', '出现', '例行', '例子', '伴随', '奥体中心', '彻底', '返回', '被', '让', '两天', '着', '有', '奥运会', '位置', '相当', '训练', '态度', '重要', '稀罕', '又', '就', '小时', '一位', '气象预报', '影响', '青睐', '无缘', '雨水', '殷家', '无奈', '导致', '症状', '比赛', '队伍', '谨慎', '已经', '造成', '长春', '给', '只', '报道', '并', '丝毫', '之前', '介绍', '要', '但', '等', '事情', '训练场', '阴沉沉', '依然', '来到', '休息', '一直', '在', '阶段', '热身赛', '现有', '这一', '抱', '之下', '后卫', '说', '记者', '受到', '事', '草草收场', '不解', '分钟', '摆脱', '上午', '出现意外', '搅和', '冯萧霆', '特别', '决定', '受凉', '嘱咐', '警惕', '中止', '表示', '可', '一路', '军', '幸好', '患上', '恢复', '工作人员', '从', '邪', '保护', '足球队', '控制', '女足', '放在', '动作', '感冒', '月', '中', '前科', '抵达', '几次', '安全', '非战斗', '起', '下午', '不断', '也', '伤病', '担心'

In [22]:
f2 = open("test.txt", "w",encoding="UTF-8")
for i in ans:
    for k in i.keys():
        for j in k: 
            f2.write(j+" ")
        f2.write('\n')

In [23]:
sentences=word2vec.Text8Corpus("test.txt")
model=gensim.models.Word2Vec(sentences,sg=1,size=100,window=5,min_count=2,negative=3,sample=0.001,hs=1,workers=4)
model.save("model")

In [24]:
model.wv.most_similar("股票",topn=10)

[('龙头', 0.8832139372825623),
 ('渠道', 0.8783321380615234),
 ('综合体', 0.8726666569709778),
 ('提问', 0.8675596714019775),
 ('老年', 0.8663349151611328),
 ('风度', 0.8658342957496643),
 ('利润', 0.8650100231170654),
 ('甚', 0.8632960319519043),
 ('怕', 0.8613040447235107),
 ('开店', 0.8607088923454285)]

In [25]:
model.wv.save_word2vec_format("output.txt")

In [27]:
print(ans[2])

{('奥帆', '透', '熟悉', '专业', '阳光', '亲吻', '项目', '人数', '到', '为', '总数', '检修', '时', '奥帆赛', '在', '将', '运动员', '已', '图', '月', '中', '日于', '目前', '休闲', '确定', '队', '级别', '有', '着', '一份', '所有', '金牌', '比基尼', '个', '规模', '正在', '下午', '参加', '性感', '块', '运动队', '均', '产生', '人', '也', '适应', '日', '比赛', '亮丽', '加紧训练', '美丽', '下', '支', '显得', '中心', '帆', '已报', '帆船', '举行', '女郎', '赛地', '青岛', '十分', '总', '日到'): 'D:/workspace/THUCNews/THUCNews/体育/2.txt'}


In [15]:
# similar=[]
# bug=[]
# for i in range(0,len(ans)):
#     for context in ans[i]:
#         try:
#             similar.append([model.most_similar(context,topn=20)])
#         except:
#             bug.append(context)
#         print ("\r Loading... "+context, end="")

In [16]:
# print(keywords[0][0][0])

In [28]:
i = 0
bug=[]
simlist=[]
similar={}
for x in rk.keys():
    try:
        tt = model.most_similar(x,topn=5)
        l = []
        for t in tt:
            simlist.append(t[0])
            l.append([t[0],float(t[1])])
        similar.setdefault(x,l)
    except:
        bug.append(context)
    print ("\r Loading... "+x,end="")

D:\Anaconda3\envs\tf2.0\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  import sys


 Loading... 费时间三条行院限公司校

In [29]:
print(len(simlist))
same=[]
for x in simlist:
    if x in rk.keys():
        same.append(x)
print(len(same))
# print(same)

25945
2147


In [30]:
index = 0
for key in similar.keys():
    index += 1
    print(key,end=" ")
    print(similar[key])
    if (index >100):
        break

国奥 [['衰落', 0.9285376071929932], ['沈总', 0.9220542907714844], ['快', 0.9202375411987305], ['待', 0.9181886911392212], ['找过', 0.9161554574966431]]
马晓旭 [['刘峰', 0.9108484983444214], ['实验舱', 0.9074091911315918], ['慢走', 0.9032933115959167], ['副省长', 0.9028279185295105], ['极佳', 0.9023455381393433]]
转好 [['尔特', 0.9183641672134399], ['十分之一', 0.912458062171936], ['旅游业', 0.9123343825340271], ['若', 0.9097244739532471], ['某类', 0.9082051515579224]]
主场优势 [['一高水', 0.9106922745704651], ['斯堪的纳维亚', 0.9081659317016602], ['杨家坪', 0.9054926633834839], ['打满', 0.9047555923461914], ['战意成', 0.9038465023040771]]
阿尔加 [['足球城', 0.9285391569137573], ['建平', 0.9238025546073914], ['高水盘', 0.9235823154449463], ['蛊惑', 0.9228324294090271], ['正餐', 0.9213758707046509]]
逼抢 [['表现', 0.9159070253372192], ['占多数', 0.9108633399009705], ['轻守', 0.9100157022476196], ['中山公园', 0.9089844226837158], ['赔上', 0.9089058637619019]]
失球 [['僵局', 0.8911672830581665], ['记录', 0.8867050409317017], ['一低', 0.8665017485618591], ['西班牙人', 0.8585937023162842], [

In [31]:
print(len(similar))

5189


In [32]:
for key in similar.keys():
    for two in similar[key]:
        if two[0] in rk.keys():
            for address in rk[two[0]]:
                flag = 1
#                 print(text[address[0]])
                for context in text[address[0]]:
#                     print(context)
                    if context[0] == two[0]:
                        flag = 0
                        break
                if flag == 1:
                    print(two[0])
                    text[address[0]].append([two[0],two[1]])
#                     text[address[0]].append(two[0])

index = 0
for key in text:
    index += 1
    print(key+':'+str(text[key]))
    if (index > 10):
        break

D:/workspace/THUCNews/THUCNews/体育/0.txt:[['国奥', 0.08812945979507042], ['马晓旭', 0.08418850354154929], ['转好', 0.08418850354154929]]
D:/workspace/THUCNews/THUCNews/体育/1.txt:[['主场优势', 0.047933371213793104], ['阿尔加', 0.047933371213793104], ['逼抢', 0.047933371213793104], ['失球', 0.045543208522068966]]
D:/workspace/THUCNews/THUCNews/体育/2.txt:[['运动队', 0.29796150692619044]]
D:/workspace/THUCNews/THUCNews/体育/3.txt:[['轮空', 0.12870997825925926], ['唐娜', 0.12870997825925926], ['女乒', 0.12870997825925926]]
D:/workspace/THUCNews/THUCNews/体育/4.txt:[]
D:/workspace/THUCNews/THUCNews/体育/5.txt:[['美国队', 0.3575538083114286], ['体育讯', 0.35117827827142856], ['外卡', 0.34156478579714283]]
D:/workspace/THUCNews/THUCNews/体育/6.txt:[['体育讯', 0.2508416273367347]]
D:/workspace/THUCNews/THUCNews/体育/7.txt:[['体育讯', 0.384101241859375]]
D:/workspace/THUCNews/THUCNews/体育/8.txt:[['队服', 0.04983973762792453], ['参透', 0.04722408789018868]]
D:/workspace/THUCNews/THUCNews/体育/9.txt:[['金牌得主', 0.06465431466046512], ['体育讯', 0.0571685569279069

In [22]:
def recommend(artical):
    try:
        for key in text[artical]:
            print(key[0])
            print(rk[key[0]])
            print()
    except:
        print("NO Such File")

In [23]:
# recommend('D:/workspace/THUCNews/THUCNews/体育/0.txt')

国奥
[['D:/workspace/THUCNews/THUCNews/体育/0.txt', 0.08812945979507042], ['D:/workspace/THUCNews/THUCNews/房产/264445.txt', 0.07275804238895349], ['D:/workspace/THUCNews/THUCNews/房产/264505.txt', 0.05793695968009259], ['D:/workspace/THUCNews/THUCNews/房产/264508.txt', 0.034474885098898074], ['D:/workspace/THUCNews/THUCNews/房产/264866.txt', 0.07192174305114943], ['D:/workspace/THUCNews/THUCNews/房产/264867.txt', 0.0839891496033557], ['D:/workspace/THUCNews/THUCNews/房产/264869.txt', 0.08342922193933333], ['D:/workspace/THUCNews/THUCNews/房产/264871.txt', 0.023304251938361265], ['D:/workspace/THUCNews/THUCNews/房产/264875.txt', 0.05875297319671361], ['D:/workspace/THUCNews/THUCNews/房产/264876.txt', 0.09932050230873016], ['D:/workspace/THUCNews/THUCNews/财经/799027.txt', 0.2406612171326923]]



In [37]:
import numpy.matlib
import pickle
userinfodic = {}
labelindex = initLabelMartix()
index = 0
for key in labelindex:
    index += 1
    print(key+':'+str(labelindex[key]))
    if (index > 10):
        break
print(len(labelindex))

with open('labelindex.txt','wb+') as f:
    pickle.dump(labelindex,f,0)

国奥:0
马晓旭:1
转好:2
主场优势:3
阿尔加:4
逼抢:5
失球:6
运动队:7
轮空:8
唐娜:9
女乒:10
5189


In [34]:
def initLabelMartix():
    labeldic = {}
    i = 0
    for label in rk.keys():
        labeldic.setdefault(label,i)
        i += 1
    return labeldic

In [116]:
def user_liked(username,artical):
    user_hobby_sum = userinfodic[username]['user_hobby_sum']
    TFIDFMartix = [0]*len(rk)
    TFMartix=[0]*len(rk)
    IDFMartix=[0]*len(rk)
    label = getLabel(artical)
    for key in label:
        user_hobby_sum[labelindex[key]] += 1
    #cal TF
    Labelsum = np.sum(user_hobby_sum)
    for i in range(0,len(TFMartix)):
        TFMartix[i] = user_hobby_sum[i]/Labelsum
    #cal IDF
    AllUserLabelSum = 0
    UserSoloLabelSum = 0
    for u in user:
        AllUserLabelSum += sum(userinfodic[u]['user_hobby_sum'])
    for i in range(0,len(IDFMartix)):
        for u in user:
            UserSoloLabelSum += userinfodic[u]['user_hobby_sum'][i]
        IDFMartix[i] = np.log(AllUserLabelSum/(UserSoloLabelSum+1))
    #cal TF*IDF
    for i in range(0,len(rk)):
        TFIDFMartix[i] = TFMartix[i]*IDFMartix[i]
    userinfodic[username]['user_hobby'] = TFIDFMartix

In [117]:
def createUser(username):
    user.append(username)
    user_hobby=[0] * len(rk)
    # print(user_hobby.shape
    user_hobby_sum=[0]*len(rk)
    print(type(user_hobby_sum))
    userinfodic.setdefault(username,{'user_hobby_sum':user_hobby_sum,'user_hobby':user_hobby})
    print("用户创建成功")
    return

In [118]:
def getLabel(artical):
    label = []
    for key in text[artical]:
        label.append(key[0])
#     print(label)
    return label
# getLabel('D:/workspace/THUCNews/THUCNews/体育/0.txt')

In [119]:
user=[]
createUser('spade')

<class 'list'>
用户创建成功


In [38]:
textindex={}
def initTextLabelList():
    textlabel={}
    length = len(rk)
#     textdic=[]
    i = 0
    for textname in text.keys():
        if not textname in textlabel.keys(): 
            textlabel.setdefault(textname,[0]*length)
            textindex.setdefault(i,textname)
            i+=1
        for x in text[textname]:
            textlabel[textname][labelindex[x[0]]] = 1
    return textlabel
textlabel = initTextLabelList()

In [43]:
textindex={}
def initTextLabelList():
    textlabel={}
    length = len(rk)
#     textdic=[]
    i = 0
    for textname in text.keys():
        if not textname in textlabel.keys(): 
            textlabel.setdefault(textname,[])
            textindex.setdefault(i,textname)
            i+=1
        for x in text[textname]:
            textlabel[textname].append(labelindex[x[0]])
    return textlabel
textlabel = initTextLabelList()

In [45]:
index = 0
for key in textlabel:
    index += 1
    print(key+':'+str(textlabel[key]))
    if (index > 10):
        break
print(len(textlabel))
with open('textlabelMartix.txt','wb+') as f:
    pickle.dump(textlabel,f,0)

D:/workspace/THUCNews/THUCNews/体育/0.txt:[0, 1, 2]
D:/workspace/THUCNews/THUCNews/体育/1.txt:[3, 4, 5, 6]
D:/workspace/THUCNews/THUCNews/体育/2.txt:[7]
D:/workspace/THUCNews/THUCNews/体育/3.txt:[8, 9, 10]
D:/workspace/THUCNews/THUCNews/体育/4.txt:[]
D:/workspace/THUCNews/THUCNews/体育/5.txt:[11, 12, 13]
D:/workspace/THUCNews/THUCNews/体育/6.txt:[12]
D:/workspace/THUCNews/THUCNews/体育/7.txt:[12]
D:/workspace/THUCNews/THUCNews/体育/8.txt:[14, 15]
D:/workspace/THUCNews/THUCNews/体育/9.txt:[16, 12]
D:/workspace/THUCNews/THUCNews/体育/10.txt:[17]
14000


In [122]:
import random
for i in range(0,100):
    number = random.randint(2000,7000)
    user_liked('spade',textindex[number])

In [124]:
import heapq
def realRecommend(user,num):
    recommendlist = []
    for x in text.keys():
        recommendlist.append(np.array(userinfodic[user]['user_hobby']).dot(np.array(textlabel[x]).T))
    res = heapq.nlargest(num, range(len(recommendlist)), recommendlist.__getitem__)
    for x in res:
        print(textindex[x])
realRecommend('spade',10)

D:/workspace/THUCNews/THUCNews/时尚/326659.txt
D:/workspace/THUCNews/THUCNews/房产/264412.txt
D:/workspace/THUCNews/THUCNews/房产/264685.txt
D:/workspace/THUCNews/THUCNews/房产/264846.txt
D:/workspace/THUCNews/THUCNews/游戏/406831.txt
D:/workspace/THUCNews/THUCNews/游戏/406848.txt
D:/workspace/THUCNews/THUCNews/游戏/406468.txt
D:/workspace/THUCNews/THUCNews/游戏/406503.txt
D:/workspace/THUCNews/THUCNews/科技/482035.txt
D:/workspace/THUCNews/THUCNews/时尚/326589.txt
